In [8]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/notebooks/meshAfterParty/meshAfterParty")

from importlib import reload

In [2]:
import datajoint as dj

In [17]:
import minfig
import datajoint_utils as du
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)


INFO:datajoint.settings:Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO:datajoint.settings:Setting enable_python_native_blobs to True


Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


In [4]:
import datajoint as dj
from minfig import adapter_objects # included with wildcard imports
minnie = dj.create_virtual_module('minnie', 'microns_minnie65_02', add_objects=adapter_objects)

schema = dj.schema("microns_minnie65_02")
dj.config["enable_python_native_blobs"] = True

Connecting celiib@at-database.ad.bcm.edu:3306


In [9]:
from importlib import reload
import os
from pathlib import Path
os.getcwd()

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)
import neuron_visualizations as nviz
import time
import system_utils as su

In [15]:
(minnie.NucleusID() & "segment_id>0") & segments.proj() 

nucleus_id id of segmented nucleus,nucleus_version segmentation version,segment_id segment_id merged to nucleus,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
3024,2,864691135087399350,57024,103536,21200,1452679,72626248513279322
3548,2,864691136303487127,63392,126240,15064,826467,73473765014505820
4068,2,864691135113166262,62576,117152,20405,878030,73402159991299630
4621,2,864691135511469390,62480,117280,21678,1178229,73331791448116869
5357,2,864691135931295367,61568,132320,19990,849311,73263484086825418
5782,2,864691135936438151,60368,137712,21009,1487117,73053065183278912
6124,2,864691136227222513,56976,132288,21413,1671738,72630165523451893
6182,2,864691134663756538,58160,134000,21374,1855385,72771109170308444
6457,2,864691135948558983,55632,147632,20084,1963268,72421052021509993
6514,2,864691135841854896,56688,154368,20251,4606505,72562751582532159


In [10]:
decimation_ratio = 0.25
decimation_version = 0

valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")
segments = (minnie.Mesh())# & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  (minnie.Decimation.proj(decimation_version='version') 
            & segments.proj() 
            & f"decimation_ratio={decimation_ratio}" 
            & f"decimation_version={decimation_version}" 
            & valid_segment_ids_with_nucleus_id)
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691135635239593,0,0.25
864691135618175635,0,0.25
864691135584809548,0,0.25
864691135679304325,0,0.25
864691136143741847,0,0.25
864691136148842647,0,0.25
864691136143284631,0,0.25
864691136745551332,0,0.25
864691136315868311,0,0.25
864691136324529047,0,0.25


In [11]:
def nucleus_id_to_seg_id(nucleus_id):
    """
    Pseudocode:
    1) restrict the nucleus id table
    2) fetch for the segment id
    """
    nucl_key = dict(nucleus_id=nucleus_id)
    nucl_seg_id = (minnie.NucleusID()  & nucl_key).fetch1("segment_id")
    if nucl_seg_id == 0:
        raise Exception(f"No segment id for nucleus Id {nucleus_id}")
    return nucl_seg_id
    
import trimesh
def fetch_segment_id_mesh(seg_id,decimation_ratio=0.25):
    key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
    new_mesh = (minnie.Decimation() & key).fetch1("mesh")
    current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
    return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)


def fetch_nucleus_id_mesh(nucleus_id,verbose=False):
    seg_id = nucleus_id_to_seg_id(nucleus_id)
    print(f"Attempting to fetch segment_id: {seg_id}")
    return fetch_segment_id_mesh(seg_id)

In [21]:
seg_ID_mesh

<trimesh.Trimesh(vertices.shape=(750183, 3), faces.shape=(1491476, 3))>

In [22]:
import trimesh_utils as tu

In [23]:
tu.write_neuron_off(seg_ID_mesh,"3_soma.off")

'3_soma.off'

In [18]:
seg_ID_mesh = fetch_segment_id_mesh(seg_ids[0])

In [19]:
nviz.plot_objects(seg_ID_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [20]:
import soma_extraction_utils as sm
soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(seg_ids[0],
                                             seg_ID_mesh.vertices,
                                             seg_ID_mesh.faces)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 6635 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/neuron_864691136819525998.off -o /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/neuron_864691136819525998_decimated.off -s /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/decimation_meshlab_25316719.mls


/usr/local/lib/python3.6/dist-packages/trimesh/grouping.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  groups = np.array(groups)
/usr/local/lib/python3.6/dist-packages/trimesh/graph.py:441: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  components = np.array([index[c] for c in components])


Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(187840, 3), faces.shape=(369170, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(187840, 3), faces.shape=(369170, 3))>
pre_largest_mesh_path = /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/neuron_864691136819525998_decimated_largest_piece.off
xvfb-run -n 3149 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/neuron_864691136819525998_decimated_largest_piece.off -o /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/neuron_864691136819525998_decimated_largest_piece_poisson.off -s /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/864691136819525998/poisson_39905.mls
Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(163322, 3), faces.shape=(326680, 3))>, <trimesh.Trimesh(vertices.shape=(51693, 3), faces.shape=(103386, 3))>]

Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 6645 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/neuron_871319.off -o /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/neuron_871319_poisson.off -s /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/poisson_386392.mls
removed temporary input file: /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/neuron_871319.off
removed temporary output file: /notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/neuron_871319_poisson.off
mesh.is_watertight = True
/notebooks/neuron_mesh_tools/test_neurons/Segmentation_2/Poisson_temp/poisson_386392.mls is being deleted....
Inside sphere validater: ratio_val = 2.8287447277758804



 Total time for run = 147.53327322006226
Before Filtering the number of somas found = 3
Performing Soma Mesh Backtracking to original mesh
-

Traceback (most recent call last):
  File "/notebooks/meshAfterParty/meshAfterParty/soma_extraction_utils.py", line 448, in extract_soma_center
    sig_th_initial_split=15)[0]
  File "/notebooks/meshAfterParty/meshAfterParty/soma_extraction_utils.py", line 882, in original_mesh_soma
    print_flag=False)
  File "/notebooks/meshAfterParty/meshAfterParty/trimesh_utils.py", line 170, in split_significant_pieces
    mesh_pieces = new_submesh.split(only_watertight=False,repair=False)
  File "/usr/local/lib/python3.6/dist-packages/trimesh/constants.py", line 134, in timed
    result = method(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/trimesh/base.py", line 1290, in split
    **kwargs)
TypeError: split() got an unexpected keyword argument 'repair'
Traceback (most recent call last):
  File "/notebooks/meshAfterParty/meshAfterParty/soma_extraction_utils.py", line 448, in extract_soma_center
    sig_th_initial_split=15)[0]
  File "/notebooks/meshAfterParty/meshAfterParty/som

In [13]:
n_verts,seg_ids = (segments & key_source & "n_vertices > 3000000").fetch("n_vertices","segment_id")
seg_ids

array([864691136819525998, 864691135707063188, 864691136265012708,
       864691135382617330, 864691135567490756, 864691135419631858,
       864691135577756100, 864691135548568516, 864691136705193838,
       864691135423873573, 864691135635256004, 864691136097234819,
       864691136700716910, 864691135574271172, 864691135435751922,
       864691135565751492, 864691135385248242, 864691135350514162,
       864691136813601902, 864691135499368644, 864691135436628978,
       864691135361593330, 864691136788552302, 864691136714471022,
       864691135432632050, 864691135433518066, 864691135556667588,
       864691135378121970, 864691135388633842, 864691135350402034,
       864691135377999602, 864691135545229252, 864691135401461746,
       864691135626318788, 864691136862154350, 864691135354391794,
       864691135455892779, 864691135579989444, 864691136813611630,
       864691135548210884, 864691135541025988, 864691135579649476,
       864691136814823022, 864691136785612142, 864691135402948